In [1]:
import json
import torch
import time
from zss import simple_distance, Node
import utils

In [2]:
#主要任务是根据策略筛选出合适的上下文，先读取数据
with open('../data/carb/carb_50_tree.json','r') as f:
    tree_50=json.load(f)
with open('../data/carb/carb_dev_tree.json','r') as f:
    tree_dev=json.load(f)

tensor_50=torch.load('../data/carb/carb_50_tensor.pkl')
tensor_dev=torch.load('../data/carb/carb_dev_tensor.pkl')

with open('../data/carb/carb_dev_tsv.txt','r') as f:
    tsv_data=f.readlines()

print(len(tree_dev),tensor_dev.size(),len(tsv_data))

633 torch.Size([633, 768]) 1266


def find_index(sent_tags,num,sent):
    num_list,word_list=[],[]
    for i in range(len(sent_tags)):
        if sent_tags[i]==num:
            word_list.append(sent[i])
            num_list.append(i)
    return num_list,word_list

with open('../../MyORE/datasets/LSOIE/dev_all.json','r') as f:
    tsv_data=json.load(f)

sent_list,triple_list=[],[]
with open('../data/LSOIE_dev_tsv.txt','w') as f:
    for data in tsv_data:
        sent=' '.join(data['sent'])
        sent_list.append(sent)
        f.write('Sentence: '+sent+'\n')
        triple=''
        for i in range(len(data['tags'])):
            sub=' '.join(data['sent'][data['a1_tags'][i][0]:data['a1_tags'][i][-1]])
            obj=' '.join(data['sent'][data['a2_tags'][i][0]:data['a2_tags'][i][-1]]) 
            _,rel=find_index(data['tags'][i],5,data['sent'])
            triple+='<'+sub+'|'+' '.join(rel)+'|'+obj+'>;'
        f.write('Triples: '+triple+'\n')
        triple_list.append(triple)
print(len(triple_list),len(sent_list))

#这儿稍微处理一下，把它变成，一行句子，第二行对应的三元组关系的txt
with open('../../openie6-master/carb/data/gold/dev.tsv','r') as f:
    tsv_data=f.readlines()
sent_list,triple_list,triple,old_sent=[],[],'',tsv_data[0].split('\t')[0]
for data in tsv_data:
    if len(data.split('\t'))>3:
        sent=data.split('\t')[0]
        if sent != old_sent:
            sent_list.append(old_sent)
            triple_list.append(triple)
            triple=''
            old_sent=sent
        rel=data.split('\t')[1]
        sub=data.split('\t')[2]
        obj=data.split('\t')[3][:-1]
        temp_triple='<'+sub+'|'+rel+'|'+obj+'>'
        if temp_triple not in triple:              #防止偶尔有特殊情况
            triple+=temp_triple+';'
    if data==tsv_data[-1]:
        sent_list.append(sent)
        triple_list.append(triple)
print(len(sent_list),len(triple_list))
#测试通过
with open('../data/carb_dev_tsv.txt','w') as f:
    for i in range(len(sent_list)):
        if '(' in sent_list[i]:
            continue
        f.write('Sentence: '+sent_list[i]+'\n')
        f.write('Triples: '+triple_list[i]+'\n')
#测试通过

In [12]:
def get_bert_similar(data_50,data_dev):                   #我想得到一个50,dev_num的张量，然后每个值都代表i，j句子的相似性
    result=torch.zeros(data_50.size()[0],data_dev.size()[0])
    for i in range(len(data_50)):
        for j in range(len(data_dev)):
            result[i,j]=torch.cosine_similarity(data_50[i],data_dev[j],dim=0)
    return result

In [3]:
def get_context(tree_dev,similar_tensor,tsv_data,topk_num=8,TED=False):
    if TED:
        prob,position=torch.topk(similar_tensor,topk_num,dim=1,largest=False)
    else:
        prob,position=torch.topk(similar_tensor,topk_num,dim=1)       #50,6的张量，position代表位置，prob代表概率！
    context=[]
    for i in range(len(position)):
        temp_context=[]
        for j in range(len(position[i])):
            sent=' '.join(tree_dev[position[i][j]]['tokens'])
            for h in range(len(tsv_data)):
                if sent==tsv_data[h][10:-1]:
                    temp_context.append(tsv_data[h]+tsv_data[h+1])
        context.append(temp_context)                       #上面是对每个position的句子找context，这个是把8个sample全找到
    return context

#print(len(tsv_data))
similar_test=get_bert_similar(tensor_50,tensor_dev)
#print(similar_test.size(),similar_test)
%time test=get_context(tree_dev,similar_test,tsv_data)
with open('../data/LSOIE/LSOIE_context_byBERT.txt','w') as f:
    for data in test:
        f.writelines(data)
        f.write('\n')
#没找到写入文件的方法，退而求其次，只能用空行来区分了

In [4]:
#上面就是基于相似性的跑完了，然后得考虑基于树的
def get_constituency_similar(tree_data,tree_dev):
    similar_tensor=torch.zeros(len(tree_data),len(tree_dev))
    TED_tensor=torch.zeros(len(tree_data),len(tree_dev))
    tree_50_list,tree_dev_list,num_50,num_dev=[],[],[],[]
    for num,data in enumerate(tree_data):
        temp_tree_50,temp_50_num=utils.get_constituency_tree(data['stanford_constituency_tree'][0])
        tree_50_list.append(temp_tree_50)
        num_50.append(temp_50_num)
        print('tree_50:',num)
    print('Tree 50 data done!')
    for num,data in enumerate(tree_dev):                  #这步很慢，得到100个树
        temp_tree_dev, temp_tree_num=utils.get_constituency_tree(data['stanford_constituency_tree'][0])#、
        tree_dev_list.append(temp_tree_dev)
        num_dev.append(temp_tree_num)
        print('tree_dev :',num)
    print('Tree dev data done!')

    for i in range(len(tree_data)):
        for j in range(len(tree_dev)):        
            edit_dis=simple_distance(tree_50_list[i], tree_dev_list[j])
            print(i,' ',j)
            TED_tensor[i][j]=edit_dis
            temp_node = num_dev[j] - num_50[i] if num_50[i]<num_dev[j] else 0.5*(num_50[i]-num_dev[j])
            tree_similiar=1-(edit_dis+temp_node)/num_50[i]
            similar_tensor[i][j]=tree_similiar
        print(i,' have done!!!')
    return similar_tensor,TED_tensor
    

In [5]:
def get_depencey_similar(tree_data,tree_dev):
    similar_tensor=torch.zeros(len(tree_data),len(tree_dev))  
    TED_tensor=torch.zeros(len(tree_data),len(tree_dev))
    tree_50_list,tree_dev_list,num_50,num_dev=[],[],[],[]
    index=0
    start_time=time.time()
    for data in tree_data:
        temp_tree_50,temp_50_num=utils.get_depencey_tree(data['dep_graph_edges'])
        index+=1
        print('tree_50 :',index)
        tree_50_list.append(temp_tree_50)
        num_50.append(temp_50_num)
    print('Tree 50 data done!',time.time()-start_time)
    index=0
    for data in tree_dev:
        temp_tree_dev,temp_dev_num=utils.get_depencey_tree(data['dep_graph_edges'])
        tree_dev_list.append(temp_tree_dev)
        num_dev.append(temp_dev_num)
        index+=1
        print('tree_dev :',index)
    print('Tree dev data done!')
    for i in range(len(tree_data)):
        start_time=time.time()
        for j in range(len(tree_dev)):        
            edit_dis=simple_distance(tree_50_list[i], tree_dev_list[j])     #这步的计算还是挺慢的说实话，一共要50*650个，一个1s的话，也得10多个小时，看看情况吧
            TED_tensor[i][j]=edit_dis
            print(i,' ',j)
            temp_node = num_dev[j] - num_50[i] if num_50[i]<num_dev[j] else 0.5*(num_50[i]-num_dev[j])
            tree_similiar=1 - (edit_dis+temp_node)/num_50[i]
            similar_tensor[i][j]=tree_similiar
        print(i,time.time()-start_time)

    return similar_tensor,TED_tensor
    

del tree_dev[480]
#carb数据集编号480的成分树过于复杂
%time test_tensor=get_constituency_similar(tree_50,tree_dev)
%time test=get_context(tree_dev,test_tensor,tsv_data)

In [ ]:
#del tree_dev[815]    
#del tree_dev[1131]   
#del tree_dev[1668]   
#本质就是LSOIE的第815和1132和1670这三个编号的数据太复杂，没法跑，把它们删了
del tree_dev[480]
%time test_tensor,TED_tensor=get_constituency_similar(tree_50,tree_dev) #这玩意跑了6个h，我服了！
%time test=get_context(tree_dev,test_tensor,tsv_data)

torch.save(test_tensor,'../data/carb/similiar_tensor_by_consti.pkl')
torch.save(TED_tensor,'../data/carb/TED_tensor_by_consti.pkl')
with open('../data/carb/carb_context_byConsti.txt','w') as f:
    for data in test:
        f.writelines(data)
        f.write('\n')


In [20]:
torch.save(bert_similar,'../data/carb/bert_similar.pkl')

In [6]:
consti=torch.load('../data/carb/TED_tensor_by_consti.pkl')
consti.size()

torch.Size([50, 632])

In [21]:
new=consti+bert_similar
new.size()

torch.Size([50, 632])

In [22]:
test=get_context(tree_dev,new,tsv_data,TED=True)
with open('../data/carb/carb_context_byBertandConsti.txt','w') as f:
    for data in test:
        f.writelines(data)
        f.write('\n')